Name: Ali Ghorbanpour - stdID: 96105994

Modern Information Retrieval

Project3 - Spring 2021


---



---



In [1]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/Colab Notebooks/

Mounted at /content/drive


# Section 1 - Crawler

Explanation

# Section 2 - Page Rank

In [ ]:
import numpy as np
import json



def page_rank(alpha):    
    with open('/content/drive/MyDrive/Colab Notebooks/CrawledPapers.json') as json_file:
        articles = json.load(json_file)

    number_of_articles = len(articles)
    articles_array = {articles[i]['id']:i for i in range(number_of_articles)}
    adjacency_array = [[0] * number_of_articles for i in range(number_of_articles)]

    for i in range(number_of_articles):
        for reference in articles[i]['references']:
            if reference in articles_array:
                adjacency_array[articles_array[reference]][i] = 1

    for i in range(number_of_articles):
        cnt = 0
        for j in range(number_of_articles):
            cnt += adjacency_array[j][i]
        for j in range(number_of_articles):
            if cnt != 0: adjacency_array[j][i] /= cnt

    for i in range(number_of_articles):
        for j in range(number_of_articles):
            adjacency_array[i][j] *= (1 - alpha)
            adjacency_array[i][j] += alpha / number_of_articles

    for i in range(100):
        u = np.matmul(adjacency_array, [1/number_of_articles] * number_of_articles)

    ans = {articles[i]['id']:u[i] for i in range(number_of_articles)}

    return ans

In [ ]:
ranked_dict = page_rank(0.5)
ranked_dict

{'2981549002\n': 0.0002500000000000028,
 '2153579005\n': 0.0002500000000000028,
 '2117130368': 0.000990873015873016,
 '2141599568': 0.0003750000000000039,
 '1662133657': 0.0005322420634920652,
 '2132339004': 0.001150892857142856,
 '2121227244': 0.0011020833333333447,
 '2194775991': 0.0007447420634920606,
 '2038721957': 0.0010098214285714229,
 '1614298861': 0.0006746031746031673,
 '1861492603': 0.0003492063492063544,
 '2952122856': 0.00027777777777778136,
 '1880262756': 0.001831051587301594,
 '1836465849': 0.0008114087301587288,
 '2097089247': 0.00039682539682539623,
 '2618530766': 0.0019484126984127166,
 '2117539524': 0.0006418650793650759,
 '2147152072': 0.002083432539682557,
 '1533861849': 0.0004755952380952397,
 '2110798204': 0.0008327380952380911,
 '2110764733': 0.0005172619047619043,
 '2167967601': 0.0002812500000000036,
 '2160660844': 0.0003819444444444467,
 '2146502635': 0.0006383928571428502,
 '2108263314': 0.0004523809523809539,
 '2155893237': 0.0007890873015872989,
 '15216262

In [ ]:
# Write output of pageRank to a json file.
jsonString = json.dumps(ranked_dict)
jsonFile = open("/content/drive/MyDrive/Colab Notebooks/PageRank.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

# Section 3 - Author ranking with HITS algorithm

Explanation

In [ ]:
import numpy as np
import json
import math


def hits(n):
    with open('/content/drive/MyDrive/Colab Notebooks/CrawledPapers.json') as json_file:
        articles = json.load(json_file)

    authors_dict = {}
    for article in articles:
        for reference in article['references']:
            for referenced_article in articles:
                if referenced_article['id'] == reference:
                    for author in article['authors']:
                        if author not in authors_dict:
                            authors_dict[author] = []
                        for referenced_author in referenced_article['authors']:
                            if referenced_author != "":
                                authors_dict[author].append(referenced_author)

    authors_dict.pop("")
    authors_map, authors_num = {}, 0
    for author in authors_dict:
        authors_map[author] = authors_num
        authors_num += 1
    authors_length = len(authors_dict)

    adjacency_matrix = [[0] * authors_length for i in range(authors_length)]
    for author in authors_dict:
        for referenced_author in authors_dict[author]:
            if referenced_author in authors_map:
                adjacency_matrix[authors_map[author]][authors_map[referenced_author]] = 1

    adjacency_matrix_transpose = np.transpose(adjacency_matrix)
    v = list(np.matmul(adjacency_matrix_transpose, [1] * authors_length))
    u = list(np.matmul(adjacency_matrix, v))

    for _ in range(5):
        values_sum = 0
        for v_s in v:
            values_sum += v_s * v_s
        values_sum = math.pow(values_sum, 1/2)
        for i in range(len(v)):
            v[i] = float(float(v[i]) / values_sum)
        values_sum = 0
        for u_s in u:
            values_sum += u_s * u_s
        values_sum = math.pow(values_sum, 1 / 2)
        for i in range(len(u)):
            u[i] = float(float(u[i]) / values_sum)

    bests = {}
    temp = list.copy(v)
    for i in range(n):
        max_value = max(temp)
        max_index = temp.index(max_value)
        for author in authors_map:
            if authors_map[author] == max_index:
                bests[author] = v[max_index]
            temp[max_index] = 0

    return bests

In [ ]:
bests = hits(5)
bests

{'Geoffrey E. Hinton': 0.2240355178286925,
 'Ilya Sutskever': 0.1661471457722854,
 'Li Fei-Fei': 0.13682706122423502,
 'Richard Socher': 0.13381987306546064,
 'Yoshua Bengio': 0.16990613097075338}

In [ ]:
jsonString = json.dumps(bests)
jsonFile = open("/content/drive/MyDrive/Colab Notebooks/HITS.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

# Section 4 - Recommender System

Explanation